# ETL Template Project

### Import necessary packages and modules

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Read the Data from CSV file into a dataframe

In [0]:
df = spark.read.csv('dbfs:/FileStore/tables/Netflix_TV_Shows_and_Movies.csv',header=True)
df.show(5)

+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|index|      id|               title| type|         description|release_year|age_certification|runtime|  imdb_id|       imdb_score|imdb_votes|
+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|    0| tm84618|         Taxi Driver|MOVIE|A mentally unstab...|        1976|                R|    113|tt0075314|              8.3|  795222.0|
|    1|tm127384|Monty Python and ...|MOVIE|"King Arthur, acc...|        1975|               PG|     91|tt0071853|8.199999999999998|  530877.0|
|    2| tm70993|       Life of Brian|MOVIE|Brian Cohen is an...|        1979|                R|     94|tt0079470|              8.0|  392419.0|
|    3|tm190788|        The Exorcist|MOVIE|12-year-old Regan...|        1973|                R|    133|tt0070047|              8.1|  391942.0|

### Convert the column data types

In [0]:
col = df.columns
colDtype = {
    'index':IntegerType(), 
    'release_year': IntegerType(), 
    'runtime': DoubleType(), 
    'imdb_score': DoubleType(), 
    'imdb_votes': DoubleType()
}
for i,j in colDtype.items():
    df = df.withColumn(i,df[i].cast(j))

df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- age_certification: string (nullable = true)
 |-- runtime: double (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- imdb_score: double (nullable = true)
 |-- imdb_votes: double (nullable = true)



### Deduplication

In [0]:
# Drop Duplicates when all the columns contains the same value
df = df.dropDuplicates()
df.show(5)

+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|index|      id|               title| type|         description|release_year|age_certification|runtime|  imdb_id|       imdb_score|imdb_votes|
+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|    2| tm70993|       Life of Brian|MOVIE|Brian Cohen is an...|        1979|                R|   94.0|tt0079470|              8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|A British sketch ...|        1969|            TV-14|   30.0|tt0063929|              8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|"King Arthur, acc...|        1975|               PG|   91.0|tt0071853|8.199999999999998|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|When a madman dub...|        1971|                R|  102.0|tt0066999|              7.7|  153463.0|

### Filtering Rows
- Drop Null Values if required
- Drop other rows based on business use case

In [0]:
## Drop if all the values in the row are null
df = df.na.drop(how='all')
df.show(5)

+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|index|      id|               title| type|         description|release_year|age_certification|runtime|  imdb_id|       imdb_score|imdb_votes|
+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|    2| tm70993|       Life of Brian|MOVIE|Brian Cohen is an...|        1979|                R|   94.0|tt0079470|              8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|A British sketch ...|        1969|            TV-14|   30.0|tt0063929|              8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|"King Arthur, acc...|        1975|               PG|   91.0|tt0071853|8.199999999999998|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|When a madman dub...|        1971|                R|  102.0|tt0066999|              7.7|  153463.0|

In [0]:
## Drop if any of the columns in a row is null
df = df.na.drop(subset=["index","id","title"])
df.show(5)

+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|index|      id|               title| type|         description|release_year|age_certification|runtime|  imdb_id|       imdb_score|imdb_votes|
+-----+--------+--------------------+-----+--------------------+------------+-----------------+-------+---------+-----------------+----------+
|    2| tm70993|       Life of Brian|MOVIE|Brian Cohen is an...|        1979|                R|   94.0|tt0079470|              8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|A British sketch ...|        1969|            TV-14|   30.0|tt0063929|              8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|"King Arthur, acc...|        1975|               PG|   91.0|tt0071853|8.199999999999998|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|When a madman dub...|        1971|                R|  102.0|tt0066999|              7.7|  153463.0|

### Filter Columns

In [0]:
df = df.drop('description')
df.show(5)

+-----+--------+--------------------+-----+------------+-----------------+-------+---------+-----------------+----------+
|index|      id|               title| type|release_year|age_certification|runtime|  imdb_id|       imdb_score|imdb_votes|
+-----+--------+--------------------+-----+------------+-----------------+-------+---------+-----------------+----------+
|    2| tm70993|       Life of Brian|MOVIE|        1979|                R|   94.0|tt0079470|              8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|        1969|            TV-14|   30.0|tt0063929|              8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|        1975|               PG|   91.0|tt0071853|8.199999999999998|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|        1971|                R|  102.0|tt0066999|              7.7|  153463.0|
|    3|tm190788|        The Exorcist|MOVIE|        1973|                R|  133.0|tt0070047|              8.1|  391942.0|
+-----+--------+--------

### Value Standardization and cleansing

In [0]:
### Round IMDB Score columns to 2 places
df = df.withColumn('imdb_score',round(df['imdb_score'],2))
df.show(5)

+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|index|      id|               title| type|release_year|age_certification|runtime|  imdb_id|imdb_score|imdb_votes|
+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|    2| tm70993|       Life of Brian|MOVIE|        1979|                R|   94.0|tt0079470|       8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|        1969|            TV-14|   30.0|tt0063929|       8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|        1975|               PG|   91.0|tt0071853|       8.2|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|        1971|                R|  102.0|tt0066999|       7.7|  153463.0|
|    3|tm190788|        The Exorcist|MOVIE|        1973|                R|  133.0|tt0070047|       8.1|  391942.0|
+-----+--------+--------------------+-----+------------+-----------------+------

In [0]:
### Replace Null values in number columns to 0
df = df.na.fill(value=0, subset=['imdb_score','imdb_votes','release_year','runtime'])
df.show(5)

+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|index|      id|               title| type|release_year|age_certification|runtime|  imdb_id|imdb_score|imdb_votes|
+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|    2| tm70993|       Life of Brian|MOVIE|        1979|                R|   94.0|tt0079470|       8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|        1969|            TV-14|   30.0|tt0063929|       8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|        1975|               PG|   91.0|tt0071853|       8.2|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|        1971|                R|  102.0|tt0066999|       7.7|  153463.0|
|    3|tm190788|        The Exorcist|MOVIE|        1973|                R|  133.0|tt0070047|       8.1|  391942.0|
+-----+--------+--------------------+-----+------------+-----------------+------

In [0]:
### Replace Null values in string column to NA
df = df.na.fill(value='NA', subset=['age_certification'])
df.show(5)

+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|index|      id|               title| type|release_year|age_certification|runtime|  imdb_id|imdb_score|imdb_votes|
+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|    2| tm70993|       Life of Brian|MOVIE|        1979|                R|   94.0|tt0079470|       8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| SHOW|        1969|            TV-14|   30.0|tt0063929|       8.8|   72895.0|
|    1|tm127384|Monty Python and ...|MOVIE|        1975|               PG|   91.0|tt0071853|       8.2|  530877.0|
|    5| tm14873|         Dirty Harry|MOVIE|        1971|                R|  102.0|tt0066999|       7.7|  153463.0|
|    3|tm190788|        The Exorcist|MOVIE|        1973|                R|  133.0|tt0070047|       8.1|  391942.0|
+-----+--------+--------------------+-----+------------+-----------------+------

In [0]:
### To lower the type value
df = df.withColumn('type',lower(df['type']))
df.show(5)

+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|index|      id|               title| type|release_year|age_certification|runtime|  imdb_id|imdb_score|imdb_votes|
+-----+--------+--------------------+-----+------------+-----------------+-------+---------+----------+----------+
|    2| tm70993|       Life of Brian|movie|        1979|                R|   94.0|tt0079470|       8.0|  392419.0|
|    4| ts22164|Monty Python's Fl...| show|        1969|            TV-14|   30.0|tt0063929|       8.8|   72895.0|
|    1|tm127384|Monty Python and ...|movie|        1975|               PG|   91.0|tt0071853|       8.2|  530877.0|
|    5| tm14873|         Dirty Harry|movie|        1971|                R|  102.0|tt0066999|       7.7|  153463.0|
|    3|tm190788|        The Exorcist|movie|        1973|                R|  133.0|tt0070047|       8.1|  391942.0|
+-----+--------+--------------------+-----+------------+-----------------+------

### Write Data to a new CSV file

In [0]:
df.coalesce(1).write.csv('dbfs:/FileStore/tables/cleansedNetflix.csv')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-86764979863222>:1
----> 1 df.coalesce(1).write.csv('dbfs:/FileStore/tables/cleansedNetflix.csv')

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1798, in DataFrameWriter.csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
   1779 self.mode(mode)
   1780 self._set_opts

In [0]:
dbutils.fs.rm('dbfs:/FileStore/tables/cleansedNetflixData.csv',True)